## 0) Import Libraries and Initialise LUSID Client

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
import globalfund as global_fund_tools
import cocoon as cocoon_tools
# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json
globals = {}

# Authenticate our user and create our API client
lusid_client = import_data.authenticate_secrets()

print('LUSID Client Initialised')
print('LUSID version : ', lusid_client.metadata.get_lusid_versions().build_version)

LUSID Client Initialised
LUSID version :  0.5.3276.0


## 1) Load Instrument Master across Regions

### Fetch the Instrument Master Data

#### US Instrument Master

In [2]:
us_instrument_master = pd.read_csv("data/global-fund-US-instrument-master.csv")

us_instrument_master.head(n=20)

,isocur,moodys_rating,name,isin,figi,assettype,issuercountry,bloombergticker,exchange_traded,couprate,int_id,sector
0,USD,NaN,Amazon_Nasdaq_AMZN,US0231351067,BBG000BVPXP1,common_stock,united_states_america,AMZN,UN,nan,imd_34634534,equity
1,USD,NaN,Apple_Nasdaq_AAPL,US0378331005,BBG000B9XVV8,common_stock,united_states_america,AAPL,UN,nan,imd_35345345,equity
2,USD,Aa2,USTreasury_2.00_2021,US912828U816,BBG00FN3B5K8,us_government,united_states_america,T 2 12/31/21,BERLIN,2.00,imd_34535347,govt
3,USD,Aa2,USTreasury_6.875_2025,US912810EV62,BBG000DQQNJ8,us_government,united_states_america,T 6.875 08/15/25,NEW YORK,6.88,imd_34534539,govt
4,USD,Baa1,TESCO_6.15_2037,US881575AC87,NaN,corporate_bond,united_states_america,NaN,NaN,6.15,imd_54001121,corporate


#### UK Instrument Master

In [3]:
uk_instrument_master = pd.read_csv("data/global-fund-UK-instrument-master.csv")

uk_instrument_master.head(n=20)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon,s&p rating
0,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,nan,NaN
1,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,nan,NaN
2,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,nan,NaN
3,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,LN,united_kingdom,GLEN,equity,common_stock,nan,NaN
4,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,LN,united_kingdom,JE/,equity,common_stock,nan,NaN
5,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,LN,united_kingdom,KGF,equity,common_stock,nan,NaN
6,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,LN,united_kingdom,MCRO,equity,common_stock,nan,NaN
7,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,LN,united_kingdom,REL,equity,common_stock,nan,NaN
8,Sage_LondonStockEx_SGE,imd_23536673,GBP,GB00B8C3BL03,BBG000BN0PP3,LN,united_kingdom,SGE,equity,common_stock,nan,NaN
9,TESCO_LondonStockEx_TSCO,imd_34634673,GBP,GB0008847096,BBG000BF46Y8,LN,united_kingdom,TSCO,equity,common_stock,nan,NaN


#### Combined Instrument Master

In [4]:
combined_instrument_master = pd.read_csv("data/global-fund-combined-instrument-master.csv")

combined_instrument_master.head(n=30)

,instrument_name,client_internal,currency,isin,figi,couprate,s&p rating,moodys_rating
0,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,nan,NaN,NaN
1,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,nan,NaN,NaN
2,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,nan,NaN,NaN
3,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,nan,NaN,NaN
4,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,nan,NaN,NaN
5,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,nan,NaN,NaN
6,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,nan,NaN,NaN
7,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,nan,NaN,NaN
8,Sage_LondonStockEx_SGE,imd_23536673,GBP,GB00B8C3BL03,BBG000BN0PP3,nan,NaN,NaN
9,TESCO_LondonStockEx_TSCO,imd_34634673,GBP,GB0008847096,BBG000BF46Y8,nan,NaN,NaN


### Load the Instruments into LUSID

In [5]:
instrument_properties_scope = 'InstrumentProperties005'

instrument_identifier_mapping = {
    'identifier_mapping': {
        "Figi": "figi",
        "Isin": "isin",
        "ClientInternal": "client_internal"
    }
}

instrument_mapping_required = {
    "name": "instrument_name"
}

instrument_mapping_optional = {}

responses = cocoon_tools.load_file_multiple_portfolios(
    client=lusid_client, 
    scope=instrument_properties_scope, 
    data_frame=combined_instrument_master, 
    mapping_required=instrument_mapping_required, 
    mapping_optional=instrument_mapping_optional, 
    source=None, 
    file_type="instrument", 
    identifier_mapping=instrument_identifier_mapping, 
    property_columns=["s&p rating", "moodys_rating", "currency"])

prettyprint.instrument_response(responses["instruments"])

Check for missing instrument properties complete
Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,Sage_LondonStockEx_SGE,imd_23536673,LUID_EWBBCKNB
1,TESCO_6.15_2037,imd_54001121,LUID_OV7OMHPD
2,USTreasury_2.00_2021,imd_34535347,LUID_YUM2HZSQ
3,TESCO_6.13_2022,imd_34001121,LUID_TD8YT3MA
4,UKGiltTreasury_3.5_2045,imd_54234532,LUID_HN4OK91W
5,UKGiltTreasury_4.5_2034,imd_34534534,LUID_LNW8I9LO
6,USTreasury_6.875_2025,imd_34534539,LUID_U65YYD85
7,Kingfisher_LondonStockEx_KGF,imd_34535552,LUID_H6LL1EIM
8,BurfordCapital_LondonStockEx_BUR,imd_43534356,LUID_EYEDZCIV
9,TESCO_LondonStockEx_TSCO,imd_34634673,LUID_UJAJVDF4


## 2) Load Transactions

### Create Transaction Portfolios

In [6]:
scopes = {
    "UK": "UK_Thinkfolio",
    "US": "US_SimcorpDimension"
}

responses = global_fund_tools.create_portfolios(
    client=lusid_client, 
    scopes=list(scopes.values()), 
    code="GlobalCreditFund", 
    currency="EUR")

for response in responses:
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: UK_Thinkfolio
Code: GlobalCreditFund
Portfolio Effective From: 2006-01-14 13:17:47.547490+00:00
Portfolio Created On: 2019-09-23 13:17:48.007459+00:00

Portfolio Created
Scope: US_SimcorpDimension
Code: GlobalCreditFund
Portfolio Effective From: 2006-01-14 13:17:47.547490+00:00
Portfolio Created On: 2019-09-23 13:17:48.413833+00:00



### Fetch the Transaction Data

#### US

In [7]:
us_transactions = pd.read_csv("data/global-fund-us-transactions.csv")

us_transactions.head(n=20)

,portfolio_code,transaction_id,instrument_name,transaction_description,transaction_type,transaction_units,transaction_price,transaction_currency,company_exposure,transaction_cost,figi,transaction_date,settlement_date,isin,mtom,accmethod,broker,region,client_internal
0,GlobalCreditFund,ab1543673,USD,Initial Funds In,FundsIn,1000000000,1,USD,NaN,1000000000,NaN,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,NaN,0,13,xy35920,CR,NaN
1,GlobalCreditFund,xfg352235,USTreasury_2.00_2021,Purchase of US Treasury Bonds,BY,50000,99,USD,USGovt,4950000,BBG00FN3B5K8,2019-09-03T18:42:12.435000+00:00,2019-09-05T18:42:12.435000+00:00,US912828U816,1,14,xy90994,CR,imd_34535347
2,GlobalCreditFund,ccv335235,USTreasury_6.875_2025,Purchase of US Treasury Bonds,BY,130000,127,USD,USGovt,16510000,BBG000DQQNJ8,2019-09-06T19:12:44.090000+00:00,2019-09-08T19:12:44.090000+00:00,US912810EV62,0,15,xy90994,CR,imd_34534539
3,GlobalCreditFund,cvcg767j31,TESCO_6.15_2037,Purchase of Tesco Corporate Bonds,BY,1750000,99,USD,TESCO,173250000,NaN,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,US881575AC87,0,21,xy90994,CR,imd_54001121


#### UK

In [8]:
uk_transactions = pd.read_csv("data/global-fund-uk-transactions.csv")

uk_transactions.head(n=20)

,desc,qty,unit_price,trade_currency,exposure,total_trade_amount,figi_identifier,trade_datetime,settlement_datetime,isin_identifier,compls,executor,location,asset_name,val,fund_code,t_id,internal_identifier
0,StartingBalance,2500000000,1,GBP,NaN,2500000000,NaN,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,NaN,MERC,jx112421,"Dublin, Ireland",GBP,532352352.00,GlobalCreditFund,kjk99898990z88908,NaN
1,Purchase,250000,149,GBP,UKGovt,37250000,BBG006N6HZM7,2019-09-03T14:42:12.435000+00:00,2019-09-05T14:42:12.435000+00:00,GB00BN65R313,SFP,jx112422,"Dublin, Ireland",UKGiltTreasury_3.5_2045,2335240000000.00,GlobalCreditFund,435opkplk03sfsf33,imd_54234532
2,Purchase,1500000,104,GBP,UKGovt,156000000,BBG001KKJLR4,2019-09-05T15:12:44.090000+00:00,2019-09-07T15:12:44.090000+00:00,GB00B4RMG977,LLK,jx112423,"Dublin, Ireland",UKGiltTreasury_3.75_2021,25252.00,GlobalCreditFund,dsfsdlkjs933234211,imd_34643653
3,Purchase,2200000,154,GBP,UKGovt,338800000,BBG0000D14P3,2019-09-10T09:58:43.112000+00:00,2019-09-11T09:58:43.112000+00:00,GB00B52WS153,LLK,jp392431,"Dublin, Ireland",UKGiltTreasury_4.5_2034,222532.00,GlobalCreditFund,dfklsjd90232350239,imd_34534534
4,Purchase,190000,108,GBP,TESCO,20520000,NaN,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,XS0414345974,MLLK,jp392431,"Luxembourg, Luxembourg",TESCO_6.13_2022,235235232.00,GlobalCreditFund,kj0llk90f0909201141,imd_34001121


#### Combined

In [9]:
combined_transactions = pd.read_csv("data/global-fund-combined-transactions.csv")

for transaction_type in ["StartingBalance", "FundsIn"]:
    combined_transactions.loc[
        combined_transactions[
            "transaction_type"] == transaction_type, 'currency_transaction'] = combined_transactions["trade_currency"]
    
combined_transactions["exchange_rate"] = 1

combined_transactions.head(n=30)

,portfolio_code,id,transaction_type,transaction_date,settlement_date,units,transaction_price,amount,trade_currency,instrument_name,...,location_region,isin,figi,client_internal,exposure_counterparty,val,compls,source,currency_transaction,exchange_rate
0,GlobalCreditFund,ab1543673,FundsIn,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,1000000000,1,1000000000,USD,USD,...,CR,NaN,NaN,NaN,NaN,nan,NaN,US_SimcorpDimension,USD,1
1,GlobalCreditFund,xfg352235,BY,2019-09-03T18:42:12.435000+00:00,2019-09-05T18:42:12.435000+00:00,50000,99,4950000,USD,USTreasury_2.00_2021,...,CR,BBG00FN3B5K8,US912828U816,imd_34535347,USGovt,nan,NaN,US_SimcorpDimension,NaN,1
2,GlobalCreditFund,ccv335235,BY,2019-09-06T19:12:44.090000+00:00,2019-09-08T19:12:44.090000+00:00,130000,127,16510000,USD,USTreasury_6.875_2025,...,CR,BBG000DQQNJ8,US912810EV62,imd_34534539,USGovt,nan,NaN,US_SimcorpDimension,NaN,1
3,GlobalCreditFund,cvcg767j31,BY,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,1750000,99,173250000,USD,TESCO_6.15_2037,...,CR,NaN,US881575AC87,imd_54001121,TESCO,nan,NaN,US_SimcorpDimension,NaN,1
4,GlobalCreditFund,kjk99898990z88908,StartingBalance,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,2500000000,1,2500000000,GBP,NaN,...,"Dublin, Ireland",NaN,NaN,NaN,NaN,532352352.00,MERC,UK_Thinkfolio,GBP,1
5,GlobalCreditFund,435opkplk03sfsf33,Purchase,2019-09-03T14:42:12.435000+00:00,2019-09-05T14:42:12.435000+00:00,250000,149,37250000,GBP,NaN,...,"Dublin, Ireland",GB00BN65R313,BBG006N6HZM7,imd_54234532,UKGovt,2335240000000.00,SFP,UK_Thinkfolio,NaN,1
6,GlobalCreditFund,dsfsdlkjs933234211,Purchase,2019-09-05T15:12:44.090000+00:00,2019-09-07T15:12:44.090000+00:00,1500000,104,156000000,GBP,NaN,...,"Dublin, Ireland",GB00B4RMG977,BBG001KKJLR4,imd_34643653,UKGovt,25252.00,LLK,UK_Thinkfolio,NaN,1
7,GlobalCreditFund,dfklsjd90232350239,Purchase,2019-09-10T09:58:43.112000+00:00,2019-09-11T09:58:43.112000+00:00,2200000,154,338800000,GBP,NaN,...,"Dublin, Ireland",GB00B52WS153,BBG0000D14P3,imd_34534534,UKGovt,222532.00,LLK,UK_Thinkfolio,NaN,1
8,GlobalCreditFund,kj0llk90f0909201141,Purchase,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,190000,108,20520000,GBP,NaN,...,"Luxembourg, Luxembourg",XS0414345974,NaN,imd_34001121,TESCO,235235232.00,MLLK,UK_Thinkfolio,NaN,1


#### Load the Transactions into LUSID

In [10]:
transaction_field_mapping_required = {
    "portfolio_code": "portfolio_code",
    "transaction_id": "id",
    "transaction_type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "units",
    "transaction_price.price": "transaction_price",
    "total_consideration.amount": "amount",
    "total_consideration.currency": "trade_currency",
    "transaction_currency": "trade_currency"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "exchange_rate"
}

transaction_identifier_mapping = {
    "identifier_mapping": {
        "Figi": "figi",
        "Isin": "isin",
        "ClientInternal": "client_internal"
    },
    "is_cash_with_currency": "currency_transaction"
}

combined_transactions = cocoon_tools.resolve_instruments(
    client=lusid_client,
    data_frame=combined_transactions,
    identifier_mapping=transaction_identifier_mapping)

for scope in list(scopes.values()):

    responses = cocoon_tools.load_file_multiple_portfolios(
        client=lusid_client, 
        scope=scope, 
        data_frame=combined_transactions.loc[
            combined_transactions["source"] == scope],
        mapping_required=transaction_field_mapping_required,
        mapping_optional=transaction_field_mapping_optional,
        source=scope,
        file_type='transaction',
        property_columns=[
            "instrument_name", 
            "accounting_method",
            "mtom",
            "broker_executor",
            "location_region",
            "exposure_counterparty",
            "val",
            "compls",
            ])

    for portfolio_code, response in responses.items():
        prettyprint.transactions_response(response, portfolio_code, scope)

Up to row 0
data types don't match for column val it is string in LUSID and float64 in file
Updated val to object


/home/jovyan/sample-notebooks/examples/use-cases/Global Fund/cocoon.py:588: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data_type_update_map[data_type_lusid], copy=False)


Check for missing transaction properties complete
Transactions Successfully Upserted into Portfolio
Scope: portfolio
Code: UK_Thinkfolio
Transactions Effective From: 2019-09-10 13:58:57.113000+00:00
Transactions Created On: 2019-09-23 13:17:51.789831+00:00

data types don't match for column accounting_method it is string in LUSID and float64 in file
Updated accounting_method to object
data types don't match for column mtom it is string in LUSID and float64 in file
Updated mtom to object
Check for missing transaction properties complete
Transactions Successfully Upserted into Portfolio
Scope: portfolio
Code: US_SimcorpDimension
Transactions Effective From: 2019-09-10 13:58:57.113000+00:00
Transactions Created On: 2019-09-23 13:17:54.667818+00:00



## 3) Configure Transaction Types

### Get Unique Types

In [11]:
combined_transactions.groupby(["transaction_type", "source"]).agg({"units": "max"})

,,units
transaction_type,source,
BY,US_SimcorpDimension,1750000
FundsIn,US_SimcorpDimension,1000000000
Purchase,UK_Thinkfolio,2200000
StartingBalance,UK_Thinkfolio,2500000000


### Create Configuration for Each Type

In [12]:
response = global_fund_tools.create_transaction_type_configuration(
    client=lusid_client, 
    aliases=[
        ("BY", "US_SimcorpDimension"),
        ("Purchase", "UK_Thinkfolio")
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types='StockMovement',
            side='Side1',
            direction=1,
            properties=None,
            mappings=None),
        models.TransactionConfigurationMovementDataRequest(
            movement_types='CashCommitment',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=None)
    ])

prettyprint.transaction_type_response(response, filters=["BY", "Purchase"])

response = global_fund_tools.create_transaction_type_configuration(
    client=lusid_client, 
    aliases=[
        ("FundsIn", "US_SimcorpDimension"),
        ("StartingBalance", "UK_Thinkfolio")
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types='CashCommitment',
            side='Side2',
            direction=1,
            properties=None,
            mappings=None)
    ])

prettyprint.transaction_type_response(response, filters=["FundsIn", "StartingBalance"])

Transaction Configuration #1

Transaction Type Aliases
Transaction Type: BY
Alias Description: PURCHASE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #20

Transaction Type Aliases
Transaction Type: BY
Alias Description: BY
Transaction Class: BY
Transaction Group: SimcorpDimension_abc
Transaction Roles: AllRoles


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #23

Transaction Type Aliases
Transaction Type: BY
Alias Description: BY
Transaction Class: BY
Transaction Group: UK_Thinkfolio
Transaction Roles: AllRoles


Transaction Type: Purchase
Alias Description: Purchase
Transaction Class: Purchase
Transaction Group: US_SimcorpDimension
Transaction Roles: AllRoles


Transaction Mo

## 4) Create Cut Labels for Region Closes

In [13]:
responses = global_fund_tools.create_cut_labels(
    client=lusid_client,
    exchange_names=["LSE", "NYSE"], 
    cut_label_type="market_close")

for response in responses:
    prettyprint.cut_label_response(response)

Cut Label Created
Display Name: LSE_market_close
Code: LSE_market_close
Local Time: 16:30
Timezone: GB
Description: LSE_market_close which is at 16:30 local time

Cut Label Created
Display Name: NYSE_market_close
Code: NYSE_market_close
Local Time: 16:00
Timezone: America/New_York
Description: NYSE_market_close which is at 16:0 local time



## 5) Load Close of Day Prices

In [14]:
marketdata = pd.read_csv("data/global-fund-marketdata.csv")

marketdata['time'] = marketdata['time'].apply(lambda x: "LSE_market_close" if x=="LSEClose" else "NYSE_market_close")
marketdata['date_cutlabel'] = marketdata['date'] + 'N' + marketdata['time']

marketdata.head(n=30)

,figi,isin,client_internal,name,currency,date,time,price,source,type,date_cutlabel
0,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,11/09/2019,LSE_market_close,106.51,bloomberg,mid_price,11/09/2019NLSE_market_close
1,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,10/09/2019,LSE_market_close,106.59,bloomberg,mid_price,10/09/2019NLSE_market_close
2,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,11/09/2019,LSE_market_close,106.52,reuters,mid_price,11/09/2019NLSE_market_close
3,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,10/09/2019,LSE_market_close,106.59,reuters,mid_price,10/09/2019NLSE_market_close
4,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,11/09/2019,LSE_market_close,151.44,bloomberg,mid_price,11/09/2019NLSE_market_close
5,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,10/09/2019,LSE_market_close,152.28,bloomberg,mid_price,10/09/2019NLSE_market_close
6,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,11/09/2019,LSE_market_close,151.22,reuters,mid_price,11/09/2019NLSE_market_close
7,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,10/09/2019,LSE_market_close,152.29,reuters,mid_price,10/09/2019NLSE_market_close
8,BBG006N6HZM7,GB00BN65R313,imd_54234532,UKGiltTreasury_3.5_2045,GBP,11/09/2019,LSE_market_close,152.44,bloomberg,mid_price,11/09/2019NLSE_market_close
9,BBG006N6HZM7,GB00BN65R313,imd_54234532,UKGiltTreasury_3.5_2045,GBP,10/09/2019,LSE_market_close,153.16,bloomberg,mid_price,10/09/2019NLSE_market_close


In [15]:
marketdata_scopes = {
    'bloomberg': 'MarketData_Bloomberg_123',
    'reuters': 'MarketData_Reuters_123'
}

instrument_identifier_mapping = {
    'identifier_mapping': {
        "Figi": "figi",
        "Isin": "isin",
        "CurrencyPair": "currency"
    }
}

instrument_identifier_heirarchy = ["Figi", "Isin", "CurrencyPair"]

quotes_mapping_required = {
    "quote_type": "type",
    "effective_at": "date_cutlabel",
    "currency": "currency",
    "value": "price"
}
    
response_1 = global_fund_tools.upsert_quotes(
    client=lusid_client,
    scope=marketdata_scopes['bloomberg'],
    data_frame=marketdata.loc[marketdata['source'] == 'bloomberg'],
    instrument_identifier_mapping=instrument_identifier_mapping, 
    instrument_identifier_heirarchy=instrument_identifier_heirarchy, 
    required_mapping=quotes_mapping_required)


response_2 = global_fund_tools.upsert_quotes(
    client=lusid_client,
    scope=marketdata_scopes['reuters'],
    data_frame=marketdata.loc[marketdata['source'] == 'reuters'],
    instrument_identifier_mapping=instrument_identifier_mapping, 
    instrument_identifier_heirarchy=instrument_identifier_heirarchy, 
    required_mapping=quotes_mapping_required)

pd.concat([response_1, response_2], ignore_index=True)

""


## 6) Group Portfolios from the US & UK

In [16]:
portfolio_group_scope = "Worldwide"

response = global_fund_tools.create_portfolio_group(
    client=lusid_client, 
    scope=portfolio_group_scope, 
    code="GlobalCreditFund", 
    portfolios=[
        models.ResourceId(
            scope=scopes["US"],
            code="GlobalCreditFund"),
        models.ResourceId(
            scope=scopes["UK"],
            code="GlobalCreditFund")
    ])

prettyprint.portfolio_group_response(response, "created")

Portfolio Group Created
Name: GlobalCreditFund
Scope: Worldwide
Code: GlobalCreditFund
Portfolios Inside Group: 
GlobalCreditFund
GlobalCreditFund




## 7) Value the GlobalCreditFund for each Region Close

### LSE Close 11th September - Bloomberg Market Data

In [17]:
global_fund_tools.valuation(
    client=lusid_client, 
    marketdata_scope=marketdata_scopes['bloomberg'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NLSE_market_close")

,Holding/default/SubHoldingKey,Instrument/default/Name,Proportion(Holding/default/PV),Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units),currency
0,Currency=USD,USD,0.20,"805,290,000.00","731,525,436.00","805,290,000.00",None
1,LusidInstrumentId=LUID_YUM2HZSQ/USD,USTreasury_2.00_2021,0.00,"4,950,000.00","4,576,973.40","50,000.00",None
2,LusidInstrumentId=LUID_U65YYD85/USD,USTreasury_6.875_2025,0.00,"16,510,000.00","15,298,818.60","130,000.00",None
3,LusidInstrumentId=LUID_OV7OMHPD/USD,TESCO_6.15_2037,0.04,"173,250,000.00","154,200,900.00","1,750,000.00",None
4,Currency=GBP,GBP,0.59,"1,947,430,000.00","2,180,342,628.00","1,947,430,000.00",None
5,LusidInstrumentId=LUID_HN4OK91W/GBP,UKGiltTreasury_3.5_2045,0.01,"37,250,000.00","42,669,355.50","250,000.00",None
6,LusidInstrumentId=LUID_06KBEF9L/GBP,UKGiltTreasury_3.75_2021,0.05,"156,000,000.00","178,872,894.00","1,500,000.00",None
7,LusidInstrumentId=LUID_LNW8I9LO/GBP,UKGiltTreasury_4.5_2034,0.10,"338,800,000.00","373,002,577.20","2,200,000.00",None
8,LusidInstrumentId=LUID_TD8YT3MA/GBP,TESCO_6.13_2022,0.01,"20,520,000.00","23,399,640.00","190,000.00",None
9,NaN,NaN,1.00,"3,500,000,000.00","3,703,889,222.70","2,758,790,000.00",NaN


### LSE Close 11th September - Reuters Market Data

In [18]:
global_fund_tools.valuation(
    client=lusid_client, 
    marketdata_scope=marketdata_scopes['reuters'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NLSE_market_close")

,Holding/default/SubHoldingKey,Instrument/default/Name,Proportion(Holding/default/PV),Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units),currency
0,Currency=USD,USD,0.20,"805,290,000.00","731,605,965.00","805,290,000.00",None
1,LusidInstrumentId=LUID_YUM2HZSQ/USD,USTreasury_2.00_2021,0.00,"4,950,000.00","4,577,931.50","50,000.00",None
2,LusidInstrumentId=LUID_U65YYD85/USD,USTreasury_6.875_2025,0.00,"16,510,000.00","15,301,683.80","130,000.00",None
3,LusidInstrumentId=LUID_OV7OMHPD/USD,TESCO_6.15_2037,0.04,"173,250,000.00","157,397,625.00","1,750,000.00",None
4,Currency=GBP,GBP,0.59,"1,947,430,000.00","2,180,537,371.00","1,947,430,000.00",None
5,LusidInstrumentId=LUID_HN4OK91W/GBP,UKGiltTreasury_3.5_2045,0.01,"37,250,000.00","42,797,453.33","250,000.00",None
6,LusidInstrumentId=LUID_06KBEF9L/GBP,UKGiltTreasury_3.75_2021,0.05,"156,000,000.00","178,907,345.55","1,500,000.00",None
7,LusidInstrumentId=LUID_LNW8I9LO/GBP,UKGiltTreasury_4.5_2034,0.10,"338,800,000.00","372,506,274.80","2,200,000.00",None
8,LusidInstrumentId=LUID_TD8YT3MA/GBP,TESCO_6.13_2022,0.01,"20,520,000.00","23,869,764.60","190,000.00",None
9,NaN,NaN,1.00,"3,500,000,000.00","3,707,501,414.58","2,758,790,000.00",NaN


### NYSE Close 11th September - Reuters Market Data

In [19]:
global_fund_tools.valuation(
    client=lusid_client, 
    marketdata_scope=marketdata_scopes['reuters'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NNYSE_market_close")

,Holding/default/SubHoldingKey,Instrument/default/Name,Proportion(Holding/default/PV),Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units),currency
0,Currency=USD,USD,0.20,"805,290,000.00","731,847,552.00","805,290,000.00",None
1,LusidInstrumentId=LUID_YUM2HZSQ/USD,USTreasury_2.00_2021,0.00,"4,950,000.00","4,579,443.20","50,000.00",None
2,LusidInstrumentId=LUID_U65YYD85/USD,USTreasury_6.875_2025,0.00,"16,510,000.00","15,306,736.64","130,000.00",None
3,LusidInstrumentId=LUID_OV7OMHPD/USD,TESCO_6.15_2037,0.04,"173,250,000.00","157,449,600.00","1,750,000.00",None
4,Currency=GBP,GBP,0.59,"1,947,430,000.00","2,176,447,768.00","1,947,430,000.00",None
5,LusidInstrumentId=LUID_HN4OK91W/GBP,UKGiltTreasury_3.5_2045,0.01,"37,250,000.00","42,717,186.60","250,000.00",None
6,LusidInstrumentId=LUID_06KBEF9L/GBP,UKGiltTreasury_3.75_2021,0.05,"156,000,000.00","178,571,804.40","1,500,000.00",None
7,LusidInstrumentId=LUID_LNW8I9LO/GBP,UKGiltTreasury_4.5_2034,0.10,"338,800,000.00","371,807,638.40","2,200,000.00",None
8,LusidInstrumentId=LUID_TD8YT3MA/GBP,TESCO_6.13_2022,0.01,"20,520,000.00","23,824,996.80","190,000.00",None
9,NaN,NaN,1.00,"3,500,000,000.00","3,702,552,726.04","2,758,790,000.00",NaN


### NYSE Close 11th September - Bloomberg Market Data

In [20]:
global_fund_tools.valuation(
    client=lusid_client, 
    marketdata_scope=marketdata_scopes['bloomberg'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NNYSE_market_close")

,Holding/default/SubHoldingKey,Instrument/default/Name,Proportion(Holding/default/PV),Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units),currency
0,Currency=USD,USD,0.20,"805,290,000.00","731,928,081.00","805,290,000.00",None
1,LusidInstrumentId=LUID_YUM2HZSQ/USD,USTreasury_2.00_2021,0.00,"4,950,000.00","4,579,492.65","50,000.00",None
2,LusidInstrumentId=LUID_U65YYD85/USD,USTreasury_6.875_2025,0.00,"16,510,000.00","15,307,239.35","130,000.00",None
3,LusidInstrumentId=LUID_OV7OMHPD/USD,TESCO_6.15_2037,0.04,"173,250,000.00","154,285,775.00","1,750,000.00",None
4,Currency=GBP,GBP,0.59,"1,947,430,000.00","2,176,837,254.00","1,947,430,000.00",None
5,LusidInstrumentId=LUID_HN4OK91W/GBP,UKGiltTreasury_3.5_2045,0.01,"37,250,000.00","42,600,755.25","250,000.00",None
6,LusidInstrumentId=LUID_06KBEF9L/GBP,UKGiltTreasury_3.75_2021,0.05,"156,000,000.00","178,585,317.00","1,500,000.00",None
7,LusidInstrumentId=LUID_LNW8I9LO/GBP,UKGiltTreasury_4.5_2034,0.10,"338,800,000.00","372,402,894.60","2,200,000.00",None
8,LusidInstrumentId=LUID_TD8YT3MA/GBP,TESCO_6.13_2022,0.01,"20,520,000.00","23,362,020.00","190,000.00",None
9,NaN,NaN,1.00,"3,500,000,000.00","3,699,888,828.85","2,758,790,000.00",NaN
